In [116]:
import numpy as np
import pandas as pd

Нас будет интересовать файл checkins.dat.

In [117]:
data = pd.read_table("checkins.dat", sep='|')

C:\Users\vlnlo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [118]:
data.rename(columns=lambda x: x.strip(), inplace=True)

Открыв его, увидим следующую структуру:

In [119]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43


In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021968 entries, 0 to 1021967
Data columns (total 6 columns):
id            1021968 non-null object
user_id       1021966 non-null float64
venue_id      1021966 non-null float64
latitude      1021966 non-null object
longitude     1021966 non-null object
created_at    1021966 non-null object
dtypes: float64(2), object(4)
memory usage: 46.8+ MB


Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [121]:
data['longitude'] = data['longitude'].apply(lambda x: float(x.strip()) if type(x) == str and bool(x.strip()) else np.nan)
data['latitude'] = data['latitude'].apply(lambda x: float(x.strip()) if type(x) == str and bool(x.strip()) else np.nan)
data = data[~data.isna()['latitude']]

data.head()

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


Убедимся, что все 396634 строки с координатами считаны успешно.

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 6 columns):
id            396634 non-null object
user_id       396634 non-null float64
venue_id      396634 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(4), object(2)
memory usage: 21.2+ MB


**Примечание** :на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [123]:
data_sub = data[:100000]

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [162]:
from sklearn.cluster import MeanShift

ms = MeanShift(bandwidth=0.1)
ms.fit(data_sub[['latitude', 'longitude']])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [153]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique, counts = np.unique(labels, return_counts=True)
n_clusters_ = len(cluster_centers)  

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [173]:
new_labels = [labels_unique[i] for i in range( len(counts) ) if counts[i] > 15]
new_cluster_centers = [cluster_centers[i] for i in range( len(counts) ) if counts[i] > 15]
new_labels, new_cluster_centers

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  15

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [186]:
offices = np.genfromtxt('centers.csv', delimiter=',')

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [217]:
distances = {}
for i in range(len(new_cluster_centers)):
    dists = np.sum((offices - new_cluster_centers[i]) ** 2, axis=1)
    min_dist = np.argmin(dists, axis=0)
    
    distances[i] = dists[min_dist]
    
distances = sorted(distances.items(), key=lambda kv: kv[1])
distances[:20]

[(398, 6.138343547438518e-05),
 (367, 8.748452367514415e-05),
 (299, 0.000499899796840809),
 (58, 0.0025058328805651507),
 (51, 0.005019401190076088),
 (25, 0.01798523282924887),
 (166, 0.028024756866587088),
 (191, 0.03479567042809154),
 (91, 0.03567412849327191),
 (85, 0.03832959557867868),
 (42, 0.04486370350982694),
 (545, 0.06658903080704684),
 (47, 0.07054666460716587),
 (55, 0.09136722465435418),
 (26, 0.09247531402178558),
 (11, 0.09915180141653804),
 (158, 0.11617575938854),
 (33, 0.13217923630202777),
 (17, 0.1434042236031941),
 (45, 0.1499520713935197)]

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [224]:
center_answer = new_cluster_centers[distances[0][0]]

In [225]:
center_answer

array([-33.86063043, 151.20477593])